In [ ]:
# Importación de bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas para mostrar más columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Bibliotecas importadas correctamente")
print("📊 Configuración de visualización establecida")


In [ ]:
# Cargar el dataset
df = pd.read_csv('../retail_sales_dataset.csv')

print("✅ Dataset cargado exitosamente")
print(f"📊 Dimensiones del dataset: {df.shape[0]} filas x {df.shape[1]} columnas")
print("\n" + "="*50)
print("📋 INFORMACIÓN BÁSICA DEL DATASET")
print("="*50)

# Información general del dataset
print(f"🔢 Número total de transacciones: {len(df):,}")
print(f"💰 Valor total de ventas: ${df['Total Amount'].sum():,.2f}")
print(f"📅 Período de datos: {df['Date'].min()} a {df['Date'].max()}")

# Mostrar primeras filas
print(f"\n📋 Primeras 5 filas del dataset:")
df.head()


In [ ]:
# Información detallada de las columnas
print("📊 INFORMACIÓN DETALLADA DE LAS COLUMNAS")
print("="*50)
print(df.info())

print("\n📈 ESTADÍSTICAS DESCRIPTIVAS")
print("="*50)
print(df.describe())


In [ ]:
# Verificar valores nulos y tipos de datos
print("🔍 ANÁLISIS DE CALIDAD DE DATOS")
print("="*50)

print("📋 Valores nulos por columna:")
missing_values = df.isnull().sum()
print(missing_values)

print(f"\n✅ Total de valores nulos: {missing_values.sum()}")

print("\n📊 Distribución de variables categóricas:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())
    print(f"Valores únicos: {df[col].nunique()}")


In [ ]:
# Crear una copia del dataset para trabajar
df_work = df.copy()

# Convertir la columna Date a datetime
df_work['Date'] = pd.to_datetime(df_work['Date'])

# Extraer características temporales
df_work['Year'] = df_work['Date'].dt.year
df_work['Month'] = df_work['Date'].dt.month
df_work['Month_Name'] = df_work['Date'].dt.strftime('%B')
df_work['Day'] = df_work['Date'].dt.day
df_work['DayOfWeek'] = df_work['Date'].dt.dayofweek
df_work['DayName'] = df_work['Date'].dt.strftime('%A')
df_work['Quarter'] = df_work['Date'].dt.quarter

# Clasificación de ventas (basado en tu trabajo anterior)
def clasificador_ventas(amount):
    if amount >= 1000:
        return 'Alta'
    elif amount >= 300:
        return 'Media'
    else:
        return 'Baja'

df_work['Sales_Category'] = df_work['Total Amount'].apply(clasificador_ventas)

# Clasificación de edad (basado en tu trabajo anterior)
def clasificador_edad(age):
    if age >= 50:
        return 'Adulto Mayor'
    elif age >= 30 and age < 50:
        return 'Adulto'
    else:
        return 'Joven'

df_work['Age_Group'] = df_work['Age'].apply(clasificador_edad)

# Normalización Min-Max (basado en tu trabajo anterior)
min_sales = df_work['Total Amount'].min()
max_sales = df_work['Total Amount'].max()
df_work['Total_Amount_Normalized'] = (df_work['Total Amount'] - min_sales) / (max_sales - min_sales)

print("✅ Feature Engineering completado")
print(f"📊 Nuevas características creadas: {len(df_work.columns) - len(df.columns)}")
print(f"📋 Columnas totales: {len(df_work.columns)}")

# Mostrar las nuevas características
print("\n🆕 Nuevas características creadas:")
new_features = set(df_work.columns) - set(df.columns)
for feature in sorted(new_features):
    print(f"   • {feature}")

df_work.head()


In [ ]:
# Seleccionar solo las columnas numéricas para el análisis de correlación
numeric_cols = df_work.select_dtypes(include=[np.number]).columns.tolist()
print(f"📊 Columnas numéricas para análisis de correlación: {len(numeric_cols)}")
print(f"📋 Variables: {', '.join(numeric_cols)}")

# Calcular la matriz de correlación
correlation_matrix = df_work[numeric_cols].corr()

# Crear el mapa de calor de correlación
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='RdYlBu_r', 
            center=0,
            square=True, 
            linewidths=0.5,
            cbar_kws={"shrink": .8},
            fmt='.2f')

plt.title('🔥 MAPA DE CALOR - MATRIZ DE CORRELACIÓN\n📊 Análisis de Relaciones entre Variables Numéricas', 
          fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Encontrar las correlaciones más fuertes
print("\n🔍 CORRELACIONES MÁS SIGNIFICATIVAS")
print("="*50)

# Crear una matriz triangular superior para evitar duplicados
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
correlation_matrix_masked = correlation_matrix.mask(mask)

# Convertir a formato largo y ordenar por valor absoluto
corr_pairs = correlation_matrix_masked.unstack().dropna()
corr_pairs_abs = corr_pairs.abs().sort_values(ascending=False)

print("💪 Top 10 correlaciones más fuertes:")
for (var1, var2), corr_value in corr_pairs_abs.head(10).items():
    original_corr = corr_pairs.loc[(var1, var2)]
    print(f"   📈 {var1} ↔ {var2}: {original_corr:.3f}")

# Análisis específico de correlaciones con Total Amount
print(f"\n💰 CORRELACIONES CON 'Total Amount':")
print("="*40)
total_amount_corr = correlation_matrix['Total Amount'].abs().sort_values(ascending=False)
for var, corr in total_amount_corr.items():
    if var != 'Total Amount':
        original_corr = correlation_matrix['Total Amount'][var]
        print(f"   📊 {var}: {original_corr:.3f}")


In [ ]:
# Función para detectar outliers usando IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Análisis de outliers para variables numéricas clave
outlier_vars = ['Total Amount', 'Age', 'Quantity', 'Price per Unit']

print("🔍 ANÁLISIS DE OUTLIERS")
print("="*50)

outlier_summary = {}

for var in outlier_vars:
    outliers, lower, upper = detect_outliers_iqr(df_work, var)
    outlier_summary[var] = {
        'count': len(outliers),
        'percentage': (len(outliers) / len(df_work)) * 100,
        'lower_bound': lower,
        'upper_bound': upper
    }
    
    print(f"\n📊 {var}:")
    print(f"   🎯 Outliers detectados: {len(outliers)} ({(len(outliers)/len(df_work)*100):.2f}%)")
    print(f"   📉 Límite inferior: {lower:.2f}")
    print(f"   📈 Límite superior: {upper:.2f}")

# Visualización de outliers con boxplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 ANÁLISIS DE OUTLIERS - BOXPLOTS\n🎯 Detección de Valores Atípicos', 
             fontsize=16, fontweight='bold', y=0.98)

for i, var in enumerate(outlier_vars):
    row = i // 2
    col = i % 2
    
    # Crear boxplot
    box_plot = axes[row, col].boxplot(df_work[var], patch_artist=True, 
                                     boxprops=dict(facecolor='lightblue', alpha=0.7),
                                     medianprops=dict(color='red', linewidth=2))
    
    axes[row, col].set_title(f'{var}\n🎯 Outliers: {outlier_summary[var]["count"]} ({outlier_summary[var]["percentage"]:.1f}%)', 
                            fontweight='bold')
    axes[row, col].set_ylabel('Valor')
    axes[row, col].grid(True, alpha=0.3)
    
    # Añadir anotaciones con estadísticas
    q1 = df_work[var].quantile(0.25)
    q3 = df_work[var].quantile(0.75)
    median = df_work[var].median()
    
    axes[row, col].annotate(f'Q1: {q1:.1f}', xy=(1.1, q1), xytext=(1.2, q1),
                           arrowprops=dict(arrowstyle='->', color='blue', alpha=0.7))
    axes[row, col].annotate(f'Mediana: {median:.1f}', xy=(1.1, median), xytext=(1.2, median),
                           arrowprops=dict(arrowstyle='->', color='red', alpha=0.7))
    axes[row, col].annotate(f'Q3: {q3:.1f}', xy=(1.1, q3), xytext=(1.2, q3),
                           arrowprops=dict(arrowstyle='->', color='blue', alpha=0.7))

plt.tight_layout()
plt.show()

# Análisis de outliers por categoría de producto
print(f"\n🏷️ OUTLIERS POR CATEGORÍA DE PRODUCTO:")
print("="*50)

for category in df_work['Product Category'].unique():
    category_data = df_work[df_work['Product Category'] == category]
    outliers, _, _ = detect_outliers_iqr(category_data, 'Total Amount')
    print(f"   📦 {category}: {len(outliers)} outliers ({len(outliers)/len(category_data)*100:.1f}%)")


In [ ]:
# 1. Análisis de Ventas por Categoría de Producto
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('📊 ANÁLISIS COMPLETO POR CATEGORÍA DE PRODUCTO\n💰 Ventas, Distribución y Patrones Temporales', 
             fontsize=18, fontweight='bold', y=0.98)

# Subplot 1: Ventas totales por categoría
category_sales = df_work.groupby('Product Category')['Total Amount'].agg(['sum', 'mean', 'count'])
bars1 = axes[0,0].bar(category_sales.index, category_sales['sum'], 
                      color=['#FF6B6B', '#4ECDC4', '#45B7D1'], alpha=0.8)
axes[0,0].set_title('💰 Ventas Totales por Categoría', fontweight='bold', fontsize=14)
axes[0,0].set_ylabel('Ventas Totales ($)')
axes[0,0].grid(True, alpha=0.3)

# Añadir valores en las barras
for bar, value in zip(bars1, category_sales['sum']):
    axes[0,0].annotate(f'${value:,.0f}', 
                       xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                       xytext=(0, 5), textcoords='offset points',
                       ha='center', va='bottom', fontweight='bold')

# Subplot 2: Distribución de ventas por categoría (Violin Plot)
violin_parts = axes[0,1].violinplot([df_work[df_work['Product Category']==cat]['Total Amount'] 
                                   for cat in df_work['Product Category'].unique()], 
                                   positions=range(len(df_work['Product Category'].unique())))
axes[0,1].set_title('🎻 Distribución de Ventas por Categoría', fontweight='bold', fontsize=14)
axes[0,1].set_ylabel('Monto de Venta ($)')
axes[0,1].set_xticks(range(len(df_work['Product Category'].unique())))
axes[0,1].set_xticklabels(df_work['Product Category'].unique())
axes[0,1].grid(True, alpha=0.3)

# Colorear los violines
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
for pc, color in zip(violin_parts['bodies'], colors):
    pc.set_facecolor(color)
    pc.set_alpha(0.7)

# Subplot 3: Ventas promedio por mes
monthly_sales = df_work.groupby('Month_Name')['Total Amount'].mean().reindex([
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
])
line1 = axes[1,0].plot(range(len(monthly_sales)), monthly_sales.values, 
                       marker='o', linewidth=3, markersize=8, color='#E74C3C')
axes[1,0].set_title('📈 Tendencia de Ventas Promedio por Mes', fontweight='bold', fontsize=14)
axes[1,0].set_ylabel('Venta Promedio ($)')
axes[1,0].set_xlabel('Mes')
axes[1,0].set_xticks(range(len(monthly_sales)))
axes[1,0].set_xticklabels([month[:3] for month in monthly_sales.index], rotation=45)
axes[1,0].grid(True, alpha=0.3)

# Añadir anotaciones para el mes con mayor y menor venta
max_month_idx = monthly_sales.values.argmax()
min_month_idx = monthly_sales.values.argmin()
axes[1,0].annotate(f'Máximo: ${monthly_sales.values[max_month_idx]:.0f}', 
                   xy=(max_month_idx, monthly_sales.values[max_month_idx]),
                   xytext=(max_month_idx, monthly_sales.values[max_month_idx] + 50),
                   arrowprops=dict(arrowstyle='->', color='green', lw=2),
                   fontweight='bold', color='green')

# Subplot 4: Heatmap de ventas por categoría y día de la semana
heatmap_data = df_work.pivot_table(values='Total Amount', 
                                  index='Product Category', 
                                  columns='DayName', 
                                  aggfunc='mean')
# Reordenar las columnas por orden de días de la semana
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data.reindex(columns=day_order)

im = axes[1,1].imshow(heatmap_data.values, cmap='YlOrRd', aspect='auto')
axes[1,1].set_title('🔥 Heatmap: Ventas Promedio por Categoría y Día', fontweight='bold', fontsize=14)
axes[1,1].set_xticks(range(len(heatmap_data.columns)))
axes[1,1].set_xticklabels([day[:3] for day in heatmap_data.columns])
axes[1,1].set_yticks(range(len(heatmap_data.index)))
axes[1,1].set_yticklabels(heatmap_data.index)

# Añadir valores en el heatmap
for i in range(len(heatmap_data.index)):
    for j in range(len(heatmap_data.columns)):
        text = axes[1,1].text(j, i, f'{heatmap_data.iloc[i, j]:.0f}',
                             ha="center", va="center", color="black", fontweight='bold')

# Añadir colorbar
plt.colorbar(im, ax=axes[1,1])

plt.tight_layout()
plt.show()


In [ ]:
# 2. Análisis Demográfico y de Comportamiento del Cliente
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('👥 ANÁLISIS DEMOGRÁFICO Y COMPORTAMIENTO DEL CLIENTE\n🎯 Patrones de Compra por Edad, Género y Categorías', 
             fontsize=18, fontweight='bold', y=0.98)

# Subplot 1: Distribución por género
gender_counts = df_work['Gender'].value_counts()
wedges, texts, autotexts = axes[0,0].pie(gender_counts.values, labels=gender_counts.index, 
                                         autopct='%1.1f%%', startangle=90,
                                         colors=['#FF9999', '#66B2FF'])
axes[0,0].set_title('👥 Distribución por Género', fontweight='bold', fontsize=14)

# Subplot 2: Ventas promedio por grupo de edad
age_sales = df_work.groupby('Age_Group')['Total Amount'].agg(['mean', 'count'])
bars2 = axes[0,1].bar(age_sales.index, age_sales['mean'], 
                      color=['#FFB84D', '#FF6B6B', '#4ECDC4'], alpha=0.8)
axes[0,1].set_title('💰 Ventas Promedio por Grupo de Edad', fontweight='bold', fontsize=14)
axes[0,1].set_ylabel('Venta Promedio ($)')
axes[0,1].grid(True, alpha=0.3)

# Añadir valores y conteos
for bar, mean_val, count in zip(bars2, age_sales['mean'], age_sales['count']):
    axes[0,1].annotate(f'${mean_val:.0f}\\n(n={count})', 
                       xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                       xytext=(0, 5), textcoords='offset points',
                       ha='center', va='bottom', fontweight='bold')

# Subplot 3: Distribución de edades
axes[0,2].hist(df_work['Age'], bins=20, color='skyblue', alpha=0.7, edgecolor='black')
axes[0,2].axvline(df_work['Age'].mean(), color='red', linestyle='--', linewidth=2, 
                  label=f'Media: {df_work["Age"].mean():.1f} años')
axes[0,2].axvline(df_work['Age'].median(), color='green', linestyle='--', linewidth=2, 
                  label=f'Mediana: {df_work["Age"].median():.1f} años')
axes[0,2].set_title('📊 Distribución de Edades', fontweight='bold', fontsize=14)
axes[0,2].set_xlabel('Edad')
axes[0,2].set_ylabel('Frecuencia')
axes[0,2].legend()
axes[0,2].grid(True, alpha=0.3)

# Subplot 4: Ventas por categoría de venta (Alta, Media, Baja)
sales_category_counts = df_work['Sales_Category'].value_counts()
bars3 = axes[1,0].bar(sales_category_counts.index, sales_category_counts.values,
                      color=['#E74C3C', '#F39C12', '#27AE60'], alpha=0.8)
axes[1,0].set_title('🎯 Distribución de Categorías de Venta', fontweight='bold', fontsize=14)
axes[1,0].set_ylabel('Número de Transacciones')
axes[1,0].grid(True, alpha=0.3)

# Añadir valores y porcentajes
total_transactions = len(df_work)
for bar, value in zip(bars3, sales_category_counts.values):
    percentage = (value / total_transactions) * 100
    axes[1,0].annotate(f'{value}\\n({percentage:.1f}%)', 
                       xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                       xytext=(0, 5), textcoords='offset points',
                       ha='center', va='bottom', fontweight='bold')

# Subplot 5: Relación entre Cantidad y Precio por Unidad
scatter = axes[1,1].scatter(df_work['Quantity'], df_work['Price per Unit'], 
                           c=df_work['Total Amount'], cmap='viridis', 
                           alpha=0.6, s=50)
axes[1,1].set_title('🔗 Relación: Cantidad vs Precio por Unidad', fontweight='bold', fontsize=14)
axes[1,1].set_xlabel('Cantidad')
axes[1,1].set_ylabel('Precio por Unidad ($)')
axes[1,1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1,1], label='Total Amount ($)')

# Subplot 6: Ventas por trimestre
quarterly_sales = df_work.groupby('Quarter')['Total Amount'].agg(['sum', 'mean'])
bars4 = axes[1,2].bar([f'Q{q}' for q in quarterly_sales.index], quarterly_sales['sum'],
                      color=['#3498DB', '#9B59B6', '#E67E22', '#1ABC9C'], alpha=0.8)
axes[1,2].set_title('📈 Ventas Totales por Trimestre', fontweight='bold', fontsize=14)
axes[1,2].set_ylabel('Ventas Totales ($)')
axes[1,2].grid(True, alpha=0.3)

# Añadir valores
for bar, value in zip(bars4, quarterly_sales['sum']):
    axes[1,2].annotate(f'${value:,.0f}', 
                       xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                       xytext=(0, 5), textcoords='offset points',
                       ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
# Resumen ejecutivo de insights clave
print("🎯 INSIGHTS CLAVE DEL ANÁLISIS EXPLORATORIO")
print("="*60)

# 1. Análisis de ventas por categoría
print("\n💰 ANÁLISIS DE VENTAS POR CATEGORÍA:")
print("-" * 40)
category_analysis = df_work.groupby('Product Category')['Total Amount'].agg(['sum', 'mean', 'count', 'std'])
for category in category_analysis.index:
    sales_total = category_analysis.loc[category, 'sum']
    sales_mean = category_analysis.loc[category, 'mean']
    transaction_count = category_analysis.loc[category, 'count']
    sales_std = category_analysis.loc[category, 'std']
    cv = sales_std / sales_mean  # Coeficiente de variación
    
    print(f"📦 {category}:")
    print(f"   💵 Ventas totales: ${sales_total:,.2f}")
    print(f"   📊 Venta promedio: ${sales_mean:.2f}")
    print(f"   🔢 Transacciones: {transaction_count}")
    print(f"   📈 Variabilidad (CV): {cv:.2f}")

# 2. Análisis temporal
print(f"\n📅 ANÁLISIS TEMPORAL:")
print("-" * 40)
best_month = df_work.groupby('Month_Name')['Total Amount'].sum().idxmax()
best_month_sales = df_work.groupby('Month_Name')['Total Amount'].sum().max()
worst_month = df_work.groupby('Month_Name')['Total Amount'].sum().idxmin()
worst_month_sales = df_work.groupby('Month_Name')['Total Amount'].sum().min()

print(f"🏆 Mejor mes: {best_month} (${best_month_sales:,.2f})")
print(f"📉 Peor mes: {worst_month} (${worst_month_sales:,.2f})")

best_day = df_work.groupby('DayName')['Total Amount'].sum().idxmax()
best_day_sales = df_work.groupby('DayName')['Total Amount'].sum().max()
print(f"🗓️ Mejor día de la semana: {best_day} (${best_day_sales:,.2f})")

# 3. Análisis demográfico
print(f"\n👥 ANÁLISIS DEMOGRÁFICO:")
print("-" * 40)
age_group_analysis = df_work.groupby('Age_Group')['Total Amount'].agg(['mean', 'count'])
print("💰 Gasto promedio por grupo de edad:")
for age_group in age_group_analysis.index:
    avg_spending = age_group_analysis.loc[age_group, 'mean']
    count = age_group_analysis.loc[age_group, 'count']
    print(f"   👤 {age_group}: ${avg_spending:.2f} (n={count})")

gender_analysis = df_work.groupby('Gender')['Total Amount'].agg(['mean', 'count'])
print(f"\n💰 Gasto promedio por género:")
for gender in gender_analysis.index:
    avg_spending = gender_analysis.loc[gender, 'mean']
    count = gender_analysis.loc[gender, 'count']
    print(f"   👤 {gender}: ${avg_spending:.2f} (n={count})")

# 4. Análisis de correlaciones clave
print(f"\n🔗 CORRELACIONES PRINCIPALES:")
print("-" * 40)
key_correlations = correlation_matrix['Total Amount'].abs().sort_values(ascending=False).head(5)
for var, corr_value in key_correlations.items():
    if var != 'Total Amount':
        original_corr = correlation_matrix['Total Amount'][var]
        print(f"   📊 {var}: {original_corr:.3f}")

# 5. Análisis de patrones de compra
print(f"\n🛒 PATRONES DE COMPRA:")
print("-" * 40)
avg_quantity = df_work['Quantity'].mean()
avg_price_per_unit = df_work['Price per Unit'].mean()
avg_total_amount = df_work['Total Amount'].mean()

print(f"📦 Cantidad promedio por transacción: {avg_quantity:.2f}")
print(f"💰 Precio promedio por unidad: ${avg_price_per_unit:.2f}")
print(f"🛍️ Monto promedio por transacción: ${avg_total_amount:.2f}")

# Distribución de categorías de venta
sales_cat_dist = df_work['Sales_Category'].value_counts(normalize=True) * 100
print(f"\n🎯 Distribución de categorías de venta:")
for category, percentage in sales_cat_dist.items():
    print(f"   📊 {category}: {percentage:.1f}%")

# 6. Recomendaciones basadas en el análisis
print(f"\n💡 RECOMENDACIONES PARA MACHINE LEARNING:")
print("-" * 50)
print("🎯 Variables más importantes para predecir ventas:")
print("   1. Quantity y Price per Unit (alta correlación)")
print("   2. Product Category (diferentes patrones de venta)")
print("   3. Características temporales (Month, DayOfWeek)")
print("   4. Age_Group (diferentes comportamientos de gasto)")
print("   5. Gender (ligeras diferencias en patrones)")

print(f"\n⚠️ CONSIDERACIONES PARA PREPROCESAMIENTO:")
print("   • Normalizar variables numéricas (diferentes escalas)")
print("   • Codificar variables categóricas")
print("   • Considerar crear features de interacción")
print("   • Evaluar tratamiento de outliers en Clothing category")

# Guardar el dataset procesado para el siguiente notebook
df_work.to_csv('../data_processed_eda.csv', index=False)
print(f"\n✅ Dataset procesado guardado como 'data_processed_eda.csv'")
print(f"📊 Listo para el siguiente paso: Preprocesamiento para Machine Learning")
